## Exploring the Historical Districts of Istanbul to Create a Historical Travel Planner

### 1. Introduction

##### 1.1 Background

Istanbul is a major city in Turkey that straddles Europe and Asia across the Bosphorus Strait. Its Old City reflects cultural influences of the many empires that once ruled here. In the Sultanahmet district, the open air , Roman-era Hippodrome was for centuries the site of chariot races, and Egyption obelisks also remain. The iconic Byzantine Hagia Sophia features a soar

##### 1.2 Problem

Travelling  to a city as historical and exciting as Istanbul can be overwhelming. Utilizing a planner can maximize 
valuable time, priotrize must-see historical places and plan for easier and faster traveln between attractions. This 
project aims to create an efficient travel planner that saves time, includes must-see attractions, and make suggestions
for what to see taking into account distance, time and location.

##### 1.3 Interest

Millions of visitors make the pilgrimage to Istanul every year to visit its historical attractions. These visitors
would be interested in the results of this project for their visit to Istanbul. Even Istanbul population can find
benefits for exploring their hometown or a new arrondissement with the results of our analysis using location data.

### 2.Data

##### 2.1 Data Source 

###### To Be Corrected

The data we will use in this project will be gathered from a variety of online sources using web scraping
techniques such as BeautifulSoup.

The arrondissements information on this Wikipedia page. Its corresponding latitude and longitude were
compiled manually using Nominatim. The top must-see Istanbul's historical attraction were found on this website as well as from Google Search, compiled with its corresponding latitude and longitude were found using Nominatim.

We will utilize a n Istanbul arrondissements geoson filr from Carto to map out the bounderies of each arronddissments and great visuals such as choropleth maps for population.The venues data was found by using the Foursquare API.

#### 2.2 Data Cleaning

##### to be Corrected

The Arrondissment that was scraped from Wikipedia site was fairly clean except for the "Arrondissment (R for Right Bank,
L for Left Bank)' column, so we added another column to the data set named "Arrondissment" that included just the numbered arrondissment. We changed the longitude and latitude columns from an object to a float type. We also added another two columns for each arrondissment's average longitude and latitude.

The famous histrical places data was compiled manually into a list and appended with its associated longitude and latitude. Utilizing Foursquare data, we downloaded the top 100 venues in each arrondissements within a 2000-meter distance from the arrondissements cooridnate. The resulting table contained the arrondissements number, longitude, latitude, venue, venue longitude longitude and latitude and venue category. Since each arrondissement's area is varried between 0.3 square miles to 3.3 square miles, there may be duplicates in our search result. We therefore dropped duplicate venues that appeared in our search.

The Istanbul geoson file was downlodaded to create arrondissements boundaried in our maps had its arrondissement name formatted in Turkish. Therefore , we created a new column that matched the format of the geojson and appended it wit longitude and latitude to be utilized in the visualizations.


In [221]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests

import json #library to handle json files

!pip install geopy
from geopy.geocoders import Nominatim # covert an adress to latitude and longitude

import requests # library to handle requests

from pandas.io.json import json_normalize # transform json file into a panada dataframe

# importing plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# importing K-Means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium #map rendering library

print('Libraries imported')

Libraries imported


In [222]:
# Blue Mosque as the main historical site and it address(from Google)
address = 'Blue Mosque, Istanbul'

geolocator = Nominatim(user_agent="Istanbul_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Geographical coordinates of Blue Mosque are {}, {}.'.format(latitude, longitude))

The Geographical coordinates of Blue Mosque are 41.0052619, 28.9768725.


In [262]:
#Foursqure Credentials and parameters
CLIENT_ID = "OLHRDU1N0XQXNQUDHDINJBDI3JOIFZRBWJ2L1T3TZHCJAWYN"
CLIENT_SECRET = "PWGBI2KUE2JIL0VNTIFFPN2EEQGA5FMGCOIWH1NGBQF51MDW"
VERSION='20180604'
LIMIT=30
radius=50000
search_query='Historic'

In [263]:
#search Foursqure 
url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}\
&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID,CLIENT_SECRET,latitude,longitude,VERSION,search_query,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OLHRDU1N0XQXNQUDHDINJBDI3JOIFZRBWJ2L1T3TZHCJAWYN&client_secret=PWGBI2KUE2JIL0VNTIFFPN2EEQGA5FMGCOIWH1NGBQF51MDW&ll=41.0052619,28.9768725&v=20180604&query=Historic&radius=50000'

In [264]:
istanbul_data=requests.get(url).json()
istanbul_data

{'meta': {'code': 200, 'requestId': '5cb7feb79fb6b7770b34429b'},
 'response': {'venues': [{'id': '4b732d5bf964a52011a02de3',
    'name': 'Galata Kulesi',
    'location': {'address': 'Bereketzade Mah. Büyük Hendek Cad. Galata',
     'lat': 41.02580757317179,
     'lng': 28.974153583057955,
     'distance': 2298,
     'postalCode': '34420',
     'cc': 'TR',
     'neighborhood': 'Bereketzade',
     'city': 'Beyoğlu',
     'state': 'İstanbul',
     'country': 'Türkiye',
     'formattedAddress': ['Bereketzade Mah. Büyük Hendek Cad. Galata',
      '34420 Beyoğlu',
      'Türkiye']},
    'categories': [{'id': '4deefb944765f83613cdba6e',
      'name': 'Historic Site',
      'pluralName': 'Historic Sites',
      'shortName': 'Historic Site',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/historicsite_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1555562167',
    'hasPerk': False},
   {'id': '4e870d4be5fa32b62a13643f',
    'name':

In [265]:
venues=istanbul_data['response']['venues']
df_istanbul= json_normalize(venues)
df_istanbul.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",False,4b732d5bf964a52011a02de3,Bereketzade Mah. Büyük Hendek Cad. Galata,TR,Beyoğlu,Türkiye,NaN,2298,"[Bereketzade Mah. Büyük Hendek Cad. Galata, 34...",NaN,41.025808,28.974154,Bereketzade,34420,İstanbul,Galata Kulesi,v-1555562167,NaN
1,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",False,4e870d4be5fa32b62a13643f,NaN,TR,NaN,Türkiye,NaN,299,[Türkiye],"[{'label': 'display', 'lat': 41.00779882226045...",41.007799,28.975671,NaN,NaN,NaN,Historical GedikPaşa Bath Turkish Hammam,v-1555562167,NaN
2,"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",False,53c50213498ec2ff5749e896,NaN,TR,İstanbul,Türkiye,Fatih,591,"[Fatih, İstanbul, Türkiye]","[{'label': 'display', 'lat': 41.00612732936857...",41.006127,28.969930,NaN,NaN,İstanbul,Historical Peninsula,v-1555562167,NaN
3,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4e23008e1495f18f035d9230,NaN,TR,İstanbul,Türkiye,NaN,1024,"[İstanbul, Türkiye]","[{'label': 'display', 'lat': 41.00436498823787...",41.004365,28.964739,NaN,NaN,İstanbul,Historical Kumkapı Restaurant,v-1555562167,415760786
4,"[{'id': '4fbc1be21983fc883593e321', 'name': 'W...",False,52c15e3b498eff91d4503a73,NaN,TR,NaN,Türkiye,NaN,7284,[Türkiye],"[{'label': 'display', 'lat': 41.0446662902832,...",41.044666,29.046116,NaN,NaN,NaN,Historic Hamam and Sauna,v-1555562167,NaN


In [266]:
df_istanbul.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.neighborhood', 'location.postalCode', 'location.state',
       'name', 'referralId', 'venuePage.id'],
      dtype='object')

In [267]:
df_istanbul_refined=df_istanbul.drop(['hasPerk','id','location.cc','location.country','location.formattedAddress',\
'location.labeledLatLngs','referralId'], axis=1)
df_istanbul_refined.columns

Index(['categories', 'location.address', 'location.city',
       'location.crossStreet', 'location.distance', 'location.lat',
       'location.lng', 'location.neighborhood', 'location.postalCode',
       'location.state', 'name', 'venuePage.id'],
      dtype='object')

In [268]:
df_istanbul_refined.head(2)

,categories,location.address,location.city,location.crossStreet,location.distance,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,venuePage.id
0,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",Bereketzade Mah. Büyük Hendek Cad. Galata,Beyoğlu,NaN,2298,41.025808,28.974154,Bereketzade,34420,İstanbul,Galata Kulesi,NaN
1,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",NaN,NaN,NaN,299,41.007799,28.975671,NaN,NaN,NaN,Historical GedikPaşa Bath Turkish Hammam,NaN


In [269]:
df_istanbul_refined_arranged=df_istanbul_refined[['name','categories','location.address',\
'location.city','location.distance','location.lat','location.lng',]]
df_istanbul_refined_arranged.head(2)

,name,categories,location.address,location.city,location.distance,location.lat,location.lng
0,Galata Kulesi,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",Bereketzade Mah. Büyük Hendek Cad. Galata,Beyoğlu,2298,41.025808,28.974154
1,Historical GedikPaşa Bath Turkish Hammam,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",NaN,NaN,299,41.007799,28.975671


In [270]:
#keep only columns that include venue name and anything is associated with location
filtered_columns=['name','categories'] + [col for col in df_istanbul_refined_arranged.columns if col.startswith('location.')]
dataframe_filtered=df_istanbul_refined_arranged.loc[:, filtered_columns]

#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

#filter category for each row
dataframe_filtered['categories'] = df_istanbul_refined_arranged.apply(get_category_type, axis=1)

#clean column names by keeping only last name
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,city,distance,lat,lng
0,Galata Kulesi,Historic Site,Bereketzade Mah. Büyük Hendek Cad. Galata,Beyoğlu,2298,41.025808,28.974154
1,Historical GedikPaşa Bath Turkish Hammam,Spa,NaN,NaN,299,41.007799,28.975671
2,Historical Peninsula,Other Great Outdoors,NaN,İstanbul,591,41.006127,28.969930
3,Historical Kumkapı Restaurant,Seafood Restaurant,NaN,İstanbul,1024,41.004365,28.964739
4,Historic Hamam and Sauna,Well,NaN,NaN,7284,41.044666,29.046116
5,Historical Preffered Hotel,Housing Development,NaN,NaN,1008,41.014194,28.978896
6,Historical Preferred Hotel Old City,Hotel,Nöbethane Cad. No:30 Sirkeci Fatih,İstanbul,1011,41.014333,28.977525
7,Historical Vezneciler Turkish Bath,Spa,Bozdogan Kemeri Cad. No. 2,İstanbul,1475,41.011254,28.961208
8,Historical Flat,Residential Building (Apartment / Condo),NaN,NaN,2232,41.025257,28.974894
9,Halat by Divan,Restaurant,Rahmi M. Koç Müzesi Hasköy Caddesi No:2 Hasköy...,İstanbul,4666,41.041223,28.948326


In [271]:
#generate map around Topkapi Serai
map_istanbul = folium.Map(location = [latitude, longitude], zoom_start = 11)
#display(map_istanbul)

#add a red circle marker to represent Topkapi Palace
folium.CircleMarker(
    [latitude, longitude],
    radius=10,color='red',
    popup='Blue Mosque',
    fill=True,fill_color='red',
    fill_opacity=0.6).add_to(map_istanbul)

#add the historical places as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat,dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5, color='blue',
        popup=label,
        fill=True, fill_color='blue',
        fill_opacity=0.6).add_to(map_istanbul)
 
   
map_istanbul